In [ ]:
# Exemplo básico de uso do pipeline Pseudo-LiDAR
print("🚀 Pseudo-LiDAR Inference Pipeline")
print("=" * 50)

print("💡 Execute as células abaixo para:")
print("  1. Instalar dependências")
print("  2. Configurar servidor SRS")
print("  3. Inicializar pipeline") 
print("  4. Executar processamento em tempo real")

print("\n🌐 Stream de saída: rtmp://195.200.0.55:1935/live/bev_processed")
print("📺 Visualizar em: http://195.200.0.55:8080/players/srs_player.html?stream=bev_processed")


In [ ]:
# Exemplo básico de uso do pipeline Pseudo-LiDAR
print("🚀 Pseudo-LiDAR Inference Pipeline")
print("=" * 50)

# Simulação de execução (descomente quando dependências estiverem instaladas)
"""
from pseudo_lidar_infer import run_pseudo_lidar_pipeline

# Executar pipeline com webcam
success = run_pseudo_lidar_pipeline(source=0, max_frames=100)

# Executar com arquivo de vídeo  
success = run_pseudo_lidar_pipeline(source="video.mp4")

# Executar com stream RTMP (mesmo do YOLO)
success = run_pseudo_lidar_pipeline(source="rtmp://195.200.0.55:1935/live/livestream")
"""

print("💡 Descomente o código acima após instalar dependências")
print("📦 Execute: pip install -r requirements.txt")
print("🌐 Stream de saída: rtmp://195.200.0.55:1935/live/bev_processed")


In [ ]:
# Instalar dependências específicas para Pseudo-LiDAR
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install transformers>=4.21.0 ultralytics>=8.0.0 opencv-python-headless>=4.7.0
%pip install ffmpeg-python requests tqdm

# Tentar instalar MiDaS (opcional)
try:
    %pip install midas
    print("✅ MiDaS instalado com sucesso!")
except:
    print("⚠️ MiDaS não disponível, usaremos apenas DPT")

# Instalar FFmpeg (necessário para streaming)
import subprocess
import sys

try:
    subprocess.run(['apt-get', 'update'], check=True, capture_output=True)
    subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], check=True, capture_output=True)
    print("✅ FFmpeg instalado com sucesso!")
except:
    print("⚠️ Erro ao instalar FFmpeg, mas continuando...")

print("✅ Setup de dependências concluído!")


In [ ]:
# Configurar path e imports do pipeline Pseudo-LiDAR
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Adicionar diretório atual ao Python path
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

print(f"📁 Diretório atual: {current_dir}")

# Importar componentes do pipeline Pseudo-LiDAR
try:
    from config import (
        CAMERA_CONFIG, BEV_CONFIG, DEPTH_MODEL, 
        SRS_SERVER_IP, OUTPUT_RTMP_URL, INPUT_SOURCE
    )
    from pipeline import PseudoLiDARPipeline, create_pipeline_from_config
    from utils import get_device, print_device_info, test_connection
    from streaming import test_streaming_server
    
    print("✅ Módulos Pseudo-LiDAR importados com sucesso!")
    
    # Verificar dispositivo
    device = get_device()
    print_device_info(device)
    
except ImportError as e:
    print(f"❌ Erro importando módulos: {e}")
    print("💡 Tentando importação alternativa...")
    
    # Importação manual como fallback
    try:
        import config
        import pipeline
        import utils
        import streaming
        
        # Re-exportar as funções necessárias
        CAMERA_CONFIG = config.CAMERA_CONFIG
        BEV_CONFIG = config.BEV_CONFIG
        DEPTH_MODEL = config.DEPTH_MODEL
        SRS_SERVER_IP = config.SRS_SERVER_IP
        OUTPUT_RTMP_URL = config.OUTPUT_RTMP_URL
        INPUT_SOURCE = config.INPUT_SOURCE
        
        PseudoLiDARPipeline = pipeline.PseudoLiDARPipeline
        create_pipeline_from_config = pipeline.create_pipeline_from_config
        get_device = utils.get_device
        print_device_info = utils.print_device_info
        test_connection = utils.test_connection
        test_streaming_server = streaming.test_streaming_server
        
        print("✅ Importação alternativa bem-sucedida!")
        
        # Verificar dispositivo
        device = get_device()
        print_device_info(device)
        
    except Exception as fallback_e:
        print(f"❌ Falha na importação alternativa: {fallback_e}")
        print("💡 Certifique-se de:")
        print("   1. Estar no diretório pseudo-lidar-infer/")
        print("   2. Todos os arquivos Python estão presentes")
        print("   3. As dependências estão instaladas")

# Verificar dependências críticas
try:
    import torch
    print(f"✅ PyTorch: {torch.__version__}")
    print(f"🔥 CUDA disponível: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
except ImportError:
    print("❌ PyTorch não instalado")

try:
    import transformers
    print(f"✅ Transformers: {transformers.__version__}")
except ImportError:
    print("❌ Transformers não instalado - necessário para DPT")

try:
    from ultralytics import YOLO
    print("✅ YOLO (Ultralytics) disponível para detecções opcionais")
except ImportError:
    print("⚠️ YOLO não instalado - detecções na BEV serão desabilitadas")

print("\\n📦 Verificação de dependências concluída!")


In [ ]:
# =============================================================================
# CONFIGURAÇÃO DO SERVIDOR SRS - COMPATÍVEL COM PROJETO YOLO
# =============================================================================

# IP do servidor (mesmo do projeto YOLO)
SRS_SERVER_IP = "195.200.0.55"  # ✅ Já configurado para servidor testado

# URLs configuradas automaticamente
SRS_RTMP_PORT = "1935"
SRS_SRT_PORT = "10080"

# URLs de streaming - DIFERENTE DO YOLO para evitar conflito
OUTPUT_RTMP_URL = f"rtmp://{SRS_SERVER_IP}:{SRS_RTMP_PORT}/live/bev_processed"  # 🆕 Nova URL
OUTPUT_SRT_URL = f"srt://{SRS_SERVER_IP}:{SRS_SRT_PORT}/bev"

# Entrada - MESMO stream do YOLO (processamento paralelo)
INPUT_SOURCE = f"rtmp://{SRS_SERVER_IP}:{SRS_RTMP_PORT}/live/livestream"  # 🔄 Mesma entrada

# OU usar outras fontes:
# INPUT_SOURCE = 0  # Para webcam
# INPUT_SOURCE = "video.mp4"  # Para arquivo de vídeo

print("🔧 Configurações Pseudo-LiDAR:")
print(f"📡 Servidor SRS: {SRS_SERVER_IP}")
print(f"📤 URL de saída BEV: {OUTPUT_RTMP_URL}")
print(f"📤 URL de saída SRT: {OUTPUT_SRT_URL}")
print(f"📹 Fonte de entrada: {INPUT_SOURCE}")

print(f"\\n🔄 Comparação com YOLO:")
print(f"   YOLO stream: rtmp://{SRS_SERVER_IP}:{SRS_RTMP_PORT}/live/processed")
print(f"   BEV stream:  {OUTPUT_RTMP_URL}")
print(f"   ✅ Podem rodar simultaneamente!")

# Testar conectividade
print(f"\\n🔍 Testando conectividade...")
if test_streaming_server(SRS_SERVER_IP, int(SRS_RTMP_PORT)):
    print("✅ Servidor SRS acessível!")
    print("🌟 Pronto para streaming Pseudo-LiDAR")
else:
    print("⚠️ Servidor SRS não acessível")
    print("💡 Pipeline funcionará sem streaming")


In [ ]:
# Criar pipeline Pseudo-LiDAR
pipeline = create_pipeline_from_config()

print("🔧 Configurações do Pipeline:")
print(f"📷 Configuração de câmera: {len(pipeline.camera_config)} parâmetros")
print(f"🗺️ Configuração BEV: {pipeline.bev_config['output_size']} @ {pipeline.bev_config['range_meters']}m")
print(f"🖥️ Dispositivo: {pipeline.device}")

# Mostrar configurações importantes
print(f"\\n📐 Parâmetros de Câmera:")
print(f"   Focal length X: {pipeline.camera_config['focal_length_x']} pixels")
print(f"   Focal length Y: {pipeline.camera_config['focal_length_y']} pixels")
print(f"   Altura da câmera: {pipeline.camera_config['camera_height_meters']} metros")

print(f"\\n🗺️ Configurações BEV:")
print(f"   Alcance: {pipeline.bev_config['range_meters']} metros")
print(f"   Resolução: {pipeline.bev_config['resolution_meters']} m/pixel")
print(f"   Tamanho de saída: {pipeline.bev_config['output_size']}")

print(f"\\n✅ Pipeline Pseudo-LiDAR criado com sucesso!")

# Carregar modelos de profundidade
print("🤖 Carregando modelos de profundidade...")

# Modelo padrão (pode ser alterado)
SELECTED_MODEL = "DPT_Large"  # Altere aqui se necessário

print(f"\\n📥 Carregando modelo: {SELECTED_MODEL}")
print("⏳ Isso pode demorar alguns minutos na primeira execução...")

if pipeline.load_models(model_name=SELECTED_MODEL):
    print(f"✅ Modelo {SELECTED_MODEL} carregado com sucesso!")
    
    # Configurar streaming
    print("📡 Configurando streaming BEV...")
    
    if pipeline.setup_streaming(OUTPUT_RTMP_URL):
        print("✅ Streaming configurado com sucesso!")
        print(f"📺 Stream BEV: {OUTPUT_RTMP_URL}")
        print(f"🌐 Player web: http://{SRS_SERVER_IP}:8080/players/srs_player.html?stream=bev_processed")
        print(f"🔗 HLS: http://{SRS_SERVER_IP}:8080/hls/live/bev_processed.m3u8")
    else:
        print("⚠️ Streaming não configurado")
        print("💡 Pipeline funcionará em modo local apenas")
        
else:
    print(f"❌ Erro ao carregar modelo {SELECTED_MODEL}")
    print("💡 Tente um modelo menor como 'MiDaS_small'")


In [ ]:
# =============================================================================
# EXECUÇÃO PRINCIPAL DO PIPELINE PSEUDO-LIDAR
# =============================================================================

# Configurações de execução
DURATION_MINUTES = 10           # Duração em minutos
MAX_FRAMES = 1000              # Máximo de frames (None = ilimitado)
ENABLE_STREAMING = True        # Se deve fazer streaming
ENABLE_YOLO_DETECTION = True   # Se deve detectar objetos na BEV

print(f"🚀 Iniciando pipeline Pseudo-LiDAR...")
print(f"⏱️ Duração: {DURATION_MINUTES} minutos ou {MAX_FRAMES} frames")
print(f"📡 Streaming: {'Habilitado' if ENABLE_STREAMING else 'Desabilitado'}")
print(f"🤖 Detecções YOLO: {'Habilitadas' if ENABLE_YOLO_DETECTION else 'Desabilitadas'}")
print(f"📹 Fonte: {INPUT_SOURCE}")
print(f"🗺️ Modelo: {SELECTED_MODEL}")

print(f"\\n📺 URLs de visualização (se streaming habilitado):")
print(f"   🌐 Player web: http://{SRS_SERVER_IP}:8080/players/srs_player.html?stream=bev_processed")
print(f"   📱 HLS: http://{SRS_SERVER_IP}:8080/hls/live/bev_processed.m3u8")

print(f"\\n⚠️ Pressione Ctrl+C para parar antecipadamente")
print(f"🔄 Processamento começando...")

# Executar pipeline
try:
    success = pipeline.run_from_config(
        source=INPUT_SOURCE,
        max_frames=MAX_FRAMES
    )
    
    if success:
        print(f"\\n🎉 Pipeline executado com sucesso!")
        
        # Mostrar estatísticas finais
        stats = pipeline.get_stats()
        print(f"\\n📊 ESTATÍSTICAS FINAIS:")
        print(f"   🎬 Frames processados: {stats.get('processed_frames', 0)}")
        print(f"   🔍 Estimativas de profundidade: {stats.get('depth_estimations', 0)}")
        print(f"   🗺️ Vistas BEV geradas: {stats.get('bev_generations', 0)}")
        print(f"   📦 Detecções totais: {stats.get('detections', 0)}")
        print(f"   ⚡ FPS médio: {stats.get('fps', 0):.1f}")
        print(f"   ⏱️ Tempo médio por frame: {stats.get('processing_time_ms', 0):.1f}ms")
        
    else:
        print(f"\\n⚠️ Pipeline interrompido ou com erro")
        print(f"💡 Verifique logs para mais detalhes")
        
except KeyboardInterrupt:
    print(f"\\n⚠️ Pipeline interrompido pelo usuário")
    
except Exception as e:
    print(f"\\n❌ Erro durante execução: {e}")
    print(f"💡 Verifique configurações e conectividade")
    
finally:
    # Limpeza
    if 'pipeline' in locals():
        pipeline.cleanup()
    print(f"🧹 Recursos limpos")


In [ ]:
# Mostrar informações do sistema
import sys
import cv2

print("🖥️ Informações do Sistema:")
print(f"   🐍 Python: {sys.version.split()[0]}")
print(f"   📷 OpenCV: {cv2.__version__}")

try:
    import torch
    print(f"   🔥 PyTorch: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"   🚀 GPU: {torch.cuda.get_device_name(0)}")
        print(f"   💾 Memória GPU Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    else:
        print("   💻 Usando CPU (sem GPU disponível)")
except:
    print(f"   ❌ PyTorch: Não instalado")

try:
    import transformers
    print(f"   🤖 Transformers: {transformers.__version__}")
except:
    print(f"   ❌ Transformers: Não instalado")

print(f"\\n📋 URLs de Acesso aos Streams:")
print(f"   🌐 Interface SRS: http://{SRS_SERVER_IP}:8080")
print(f"   📺 Player BEV: http://{SRS_SERVER_IP}:8080/players/srs_player.html?stream=bev_processed")
print(f"   📺 Player YOLO: http://{SRS_SERVER_IP}:8080/players/srs_player.html?stream=processed")
print(f"   📱 HLS BEV: http://{SRS_SERVER_IP}:8080/hls/live/bev_processed.m3u8")
print(f"   📱 HLS YOLO: http://{SRS_SERVER_IP}:8080/hls/live/processed.m3u8")

print(f"\\n🔄 Comparação de Streams:")
print(f"   YOLO: rtmp://{SRS_SERVER_IP}:1935/live/processed (detecções 2D)")
print(f"   BEV:  rtmp://{SRS_SERVER_IP}:1935/live/bev_processed (vista 3D)")

print(f"\\n📊 Performance Esperada:")
print(f"   🎯 YOLO: 25+ FPS, 44MB GPU, detecções rápidas")
print(f"   🗺️ Pseudo-LiDAR: 15 FPS, 1-2GB GPU, mapeamento 3D")
print(f"   ✅ Podem executar simultaneamente!")

print(f"\\n🎮 Como usar:")
print(f"   1. Execute células de configuração e inicialização")
print(f"   2. Execute célula de 'Execução Principal'")
print(f"   3. Acesse URLs acima para visualizar streams")
print(f"   4. Compare com stream YOLO se disponível")


In [ ]:
# Exemplo básico de uso do pipeline Pseudo-LiDAR
print("🚀 Pseudo-LiDAR Inference Pipeline")
print("=" * 50)

# Simulação de execução (descomente quando dependências estiverem instaladas)
"""
from pseudo_lidar_infer import run_pseudo_lidar_pipeline

# Executar pipeline com webcam
success = run_pseudo_lidar_pipeline(source=0, max_frames=100)

# Executar com arquivo de vídeo  
success = run_pseudo_lidar_pipeline(source="video.mp4")

# Executar com stream RTMP (mesmo do YOLO)
success = run_pseudo_lidar_pipeline(source="rtmp://195.200.0.55:1935/live/livestream")
"""

print("💡 Descomente o código acima após instalar dependências")
print("📦 Execute: pip install -r requirements.txt")
print("🌐 Stream de saída: rtmp://195.200.0.55:1935/live/bev_processed")


In [ ]:
# Instalar dependências específicas para Pseudo-LiDAR
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install transformers>=4.21.0 ultralytics>=8.0.0 opencv-python-headless>=4.7.0
%pip install ffmpeg-python requests tqdm

# Tentar instalar MiDaS (opcional)
try:
    %pip install midas
    print("✅ MiDaS instalado com sucesso!")
except:
    print("⚠️ MiDaS não disponível, usaremos apenas DPT")

# Instalar FFmpeg (necessário para streaming)
import subprocess
import sys

try:
    subprocess.run(['apt-get', 'update'], check=True, capture_output=True)
    subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], check=True, capture_output=True)
    print("✅ FFmpeg instalado com sucesso!")
except:
    print("⚠️ Erro ao instalar FFmpeg, mas continuando...")

print("✅ Setup de dependências concluído!")


In [ ]:
# Importar componentes do pipeline Pseudo-LiDAR
try:
    from config import (
        CAMERA_CONFIG, BEV_CONFIG, DEPTH_MODEL, 
        SRS_SERVER_IP, OUTPUT_RTMP_URL, INPUT_SOURCE
    )
    from pipeline import PseudoLiDARPipeline, create_pipeline_from_config
    from utils import get_device, print_device_info, test_connection
    from streaming import test_streaming_server
    
    print("✅ Módulos Pseudo-LiDAR importados com sucesso!")
    
except ImportError as e:
    print(f"❌ Erro importando módulos: {e}")
    print("💡 Certifique-se de estar no diretório pseudo-lidar-infer/")
    print("💡 Execute: cd pseudo-lidar-infer")

# Imports padrão
import warnings
warnings.filterwarnings('ignore')

# Verificar dispositivo
device = get_device()
print_device_info(device)

# Verificar dependências críticas
try:
    import torch
    print(f"✅ PyTorch: {torch.__version__}")
    print(f"🔥 CUDA disponível: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
except ImportError:
    print("❌ PyTorch não instalado")

try:
    import transformers
    print(f"✅ Transformers: {transformers.__version__}")
except ImportError:
    print("❌ Transformers não instalado - necessário para DPT")

try:
    from ultralytics import YOLO
    print("✅ YOLO (Ultralytics) disponível para detecções opcionais")
except ImportError:
    print("⚠️ YOLO não instalado - detecções na BEV serão desabilitadas")

print("\n📦 Verificação de dependências concluída!")


In [ ]:
# =============================================================================
# CONFIGURAÇÃO DO SERVIDOR SRS - COMPATÍVEL COM PROJETO YOLO
# =============================================================================

# IP do servidor (mesmo do projeto YOLO)
SRS_SERVER_IP = "195.200.0.55"  # ✅ Já configurado para servidor testado

# URLs configuradas automaticamente
SRS_RTMP_PORT = "1935"
SRS_SRT_PORT = "10080"

# URLs de streaming - DIFERENTE DO YOLO para evitar conflito
OUTPUT_RTMP_URL = f"rtmp://{SRS_SERVER_IP}:{SRS_RTMP_PORT}/live/bev_processed"  # 🆕 Nova URL
OUTPUT_SRT_URL = f"srt://{SRS_SERVER_IP}:{SRS_SRT_PORT}/bev"

# Entrada - MESMO stream do YOLO (processamento paralelo)
INPUT_SOURCE = f"rtmp://{SRS_SERVER_IP}:{SRS_RTMP_PORT}/live/livestream"  # 🔄 Mesma entrada

# OU usar outras fontes:
# INPUT_SOURCE = 0  # Para webcam
# INPUT_SOURCE = "video.mp4"  # Para arquivo de vídeo

print("🔧 Configurações Pseudo-LiDAR:")
print(f"📡 Servidor SRS: {SRS_SERVER_IP}")
print(f"📤 URL de saída BEV: {OUTPUT_RTMP_URL}")
print(f"📤 URL de saída SRT: {OUTPUT_SRT_URL}")
print(f"📹 Fonte de entrada: {INPUT_SOURCE}")

print(f"\n🔄 Comparação com YOLO:")
print(f"   YOLO stream: rtmp://{SRS_SERVER_IP}:{SRS_RTMP_PORT}/live/processed")
print(f"   BEV stream:  {OUTPUT_RTMP_URL}")
print(f"   ✅ Podem rodar simultaneamente!")

# Testar conectividade
print(f"\n🔍 Testando conectividade...")
if test_streaming_server(SRS_SERVER_IP, int(SRS_RTMP_PORT)):
    print("✅ Servidor SRS acessível!")
    print("🌟 Pronto para streaming Pseudo-LiDAR")
else:
    print("⚠️ Servidor SRS não acessível")
    print("💡 Pipeline funcionará sem streaming")


In [ ]:
# =============================================================================
# APLICAR CONFIGURAÇÕES CUSTOMIZADAS AO MÓDULO
# =============================================================================

# Importar e atualizar as configurações do módulo
try:
    import config as config_module
    
    # Atualizar as configurações no módulo importado
    config_module.SRS_SERVER_IP = SRS_SERVER_IP
    config_module.SRS_RTMP_PORT = SRS_RTMP_PORT
    config_module.SRS_SRT_PORT = SRS_SRT_PORT
    config_module.INPUT_SOURCE = INPUT_SOURCE
    config_module.OUTPUT_RTMP_URL = OUTPUT_RTMP_URL
    config_module.OUTPUT_SRT_URL = OUTPUT_SRT_URL
    
    print("✅ Configurações aplicadas ao módulo!")
    print(f"🔧 IP do servidor configurado: {config_module.SRS_SERVER_IP}")
    print(f"🔧 URL de saída BEV: {config_module.OUTPUT_RTMP_URL}")
    print(f"🔧 Entrada configurada: {config_module.INPUT_SOURCE}")
    
except Exception as e:
    print(f"⚠️ Erro aplicando configurações: {e}")
    print("💡 Usando configurações padrão do módulo")


In [ ]:
# Criar pipeline Pseudo-LiDAR
pipeline = create_pipeline_from_config()

print("🔧 Configurações do Pipeline:")
print(f"📷 Configuração de câmera: {len(pipeline.camera_config)} parâmetros")
print(f"🗺️ Configuração BEV: {pipeline.bev_config['output_size']} @ {pipeline.bev_config['range_meters']}m")
print(f"🖥️ Dispositivo: {pipeline.device}")

# Mostrar configurações importantes
print(f"\n📐 Parâmetros de Câmera:")
print(f"   Focal length X: {pipeline.camera_config['focal_length_x']} pixels")
print(f"   Focal length Y: {pipeline.camera_config['focal_length_y']} pixels")
print(f"   Altura da câmera: {pipeline.camera_config['camera_height_meters']} metros")

print(f"\n🗺️ Configurações BEV:")
print(f"   Alcance: {pipeline.bev_config['range_meters']} metros")
print(f"   Resolução: {pipeline.bev_config['resolution_meters']} m/pixel")
print(f"   Tamanho de saída: {pipeline.bev_config['output_size']}")

print(f"\n✅ Pipeline Pseudo-LiDAR criado com sucesso!")


In [ ]:
# Carregar modelos de profundidade
print("🤖 Carregando modelos de profundidade...")

# Modelos disponíveis
AVAILABLE_MODELS = {
    "DPT_Large": "Máxima qualidade, mais lento (1-2GB GPU)",
    "DPT_Hybrid": "Balanceado (800MB-1.2GB GPU)", 
    "MiDaS": "Rápido, boa qualidade (500MB-800MB GPU)",
    "MiDaS_small": "Muito rápido, qualidade adequada (200-400MB GPU)"
}

print("📋 Modelos disponíveis:")
for model, desc in AVAILABLE_MODELS.items():
    print(f"   {model}: {desc}")

# Modelo padrão (pode ser alterado)
SELECTED_MODEL = "DPT_Large"  # Altere aqui se necessário

print(f"\n📥 Carregando modelo: {SELECTED_MODEL}")
print("⏳ Isso pode demorar alguns minutos na primeira execução...")

if pipeline.load_models(model_name=SELECTED_MODEL):
    print(f"✅ Modelo {SELECTED_MODEL} carregado com sucesso!")
    print(f"💾 Uso de memória: {pipeline._get_memory_usage()}")
    
    # Verificar se YOLO foi carregado para detecções
    if hasattr(pipeline.depth_processor, 'yolo_model') and pipeline.depth_processor.yolo_model:
        print("✅ YOLO carregado para detecções na BEV")
    else:
        print("ℹ️ YOLO não carregado - detecções desabilitadas")
        
else:
    print(f"❌ Erro ao carregar modelo {SELECTED_MODEL}")
    print("💡 Tente um modelo menor como 'MiDaS_small'")
    print("💡 Ou verifique se as dependências estão instaladas")


In [ ]:
# Configurar streaming
print("📡 Configurando streaming BEV...")

if pipeline.setup_streaming(OUTPUT_RTMP_URL):
    print("✅ Streaming configurado com sucesso!")
    print(f"📺 Stream BEV: {OUTPUT_RTMP_URL}")
    print(f"🌐 Player web: http://{SRS_SERVER_IP}:8080/players/srs_player.html?stream=bev_processed")
    print(f"🔗 HLS: http://{SRS_SERVER_IP}:8080/hls/live/bev_processed.m3u8")
else:
    print("⚠️ Streaming não configurado")
    print("💡 Pipeline funcionará em modo local apenas")
    print("💡 Verifique conectividade com servidor SRS")

print(f"\n🎯 Pipeline Pseudo-LiDAR pronto para uso!")
print(f"📊 Configuração: {SELECTED_MODEL} + BEV + Streaming")
print(f"🔄 Resolução final: 1920x480 (Original|Profundidade|BEV)")

# Mostrar diferença com YOLO
print(f"\n🆚 Comparação com pipeline YOLO:")
print(f"   YOLO: Detecções 2D rápidas (25+ FPS)")
print(f"   Pseudo-LiDAR: Vista BEV 3D (15 FPS, maior precisão)")
print(f"   ✅ Ambos podem rodar simultaneamente no mesmo servidor!")


In [ ]:
# Função para criar/carregar imagem de teste
def load_test_image():
    """Carrega imagem de teste ou cria uma sintética."""
    import numpy as np
    import cv2
    import os
    
    # Tentar carregar imagem de teste
    test_paths = [
        '../yolo-infer/test_image.jpg',
        'test_image.jpg',
        'sample.jpg'
    ]
    
    for path in test_paths:
        if os.path.exists(path):
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            print(f"✅ Imagem carregada: {path}")
            return cv2.resize(img, (640, 480))
    
    # Criar imagem sintética se não encontrar
    print("📝 Criando imagem sintética de teste")
    img = np.zeros((480, 640, 3), dtype=np.uint8)
    
    # Simular uma cena
    img[300:480, :] = [70, 70, 70]      # Estrada cinza
    img[200:300, :] = [34, 139, 34]     # Grama verde  
    img[:200, :] = [135, 206, 235]      # Céu azul
    
    # Adicionar alguns "objetos"
    cv2.rectangle(img, (150, 320), (230, 400), (255, 0, 0), -1)    # Carro vermelho
    cv2.rectangle(img, (350, 340), (430, 420), (0, 0, 255), -1)    # Carro azul
    cv2.rectangle(img, (500, 280), (520, 380), (0, 255, 0), -1)    # Pessoa verde
    
    return img

# Testar modelo com imagem
print("🧪 Executando teste com imagem...")

if pipeline.depth_processor:
    # Carregar imagem de teste
    test_image = load_test_image()
    
    if test_image is not None:
        print(f"📏 Dimensões da imagem: {test_image.shape}")
        
        # Processar imagem
        print("🔄 Processando com pipeline Pseudo-LiDAR...")
        
        # Converter para BGR (OpenCV format)
        test_image_bgr = cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR)
        
        # Processar
        import time
        start_time = time.time()
        results = pipeline.depth_processor.process_frame(test_image_bgr, enable_detection=True)
        processing_time = time.time() - start_time
        
        if results['success']:
            print(f"✅ Teste concluído com sucesso!")
            print(f"⏱️ Tempo de processamento: {processing_time:.2f}s")
            print(f"🔍 Detecções encontradas: {results['detections']}")
            print(f"🗺️ BEV gerada: {results['bev_image'].shape if results['bev_image'] is not None else 'Falhou'}")
            
            # Salvar resultado (se possível)
            try:
                if results['processed_bev'] is not None:
                    cv2.imwrite('test_bev_result.jpg', results['processed_bev'])
                    print("📸 Resultado salvo: test_bev_result.jpg")
            except:
                print("⚠️ Não foi possível salvar resultado")
                
        else:
            print("❌ Erro no processamento")
            print("💡 Verifique se os modelos foram carregados corretamente")
    else:
        print("❌ Não foi possível carregar/criar imagem de teste")
        
else:
    print("❌ Pipeline não inicializado")
    print("💡 Execute as células de inicialização primeiro")


In [ ]:
# =============================================================================
# EXECUÇÃO PRINCIPAL DO PIPELINE PSEUDO-LIDAR
# =============================================================================

# Configurações de execução
DURATION_MINUTES = 10           # Duração em minutos
MAX_FRAMES = 1000              # Máximo de frames (None = ilimitado)
ENABLE_STREAMING = True        # Se deve fazer streaming
ENABLE_YOLO_DETECTION = True   # Se deve detectar objetos na BEV

print(f"🚀 Iniciando pipeline Pseudo-LiDAR...")
print(f"⏱️ Duração: {DURATION_MINUTES} minutos ou {MAX_FRAMES} frames")
print(f"📡 Streaming: {'Habilitado' if ENABLE_STREAMING else 'Desabilitado'}")
print(f"🤖 Detecções YOLO: {'Habilitadas' if ENABLE_YOLO_DETECTION else 'Desabilitadas'}")
print(f"📹 Fonte: {INPUT_SOURCE}")
print(f"🗺️ Modelo: {SELECTED_MODEL}")

print(f"\n📊 Expectativa de Performance:")
if device == 'cuda':
    if SELECTED_MODEL == "DPT_Large":
        print(f"   🚀 FPS esperado: 10-15 FPS (GPU)")
    elif SELECTED_MODEL == "MiDaS":
        print(f"   🚀 FPS esperado: 20-25 FPS (GPU)")
    else:
        print(f"   🚀 FPS esperado: 15-20 FPS (GPU)")
else:
    print(f"   💻 FPS esperado: 2-8 FPS (CPU)")

print(f"\n📺 URLs de visualização (se streaming habilitado):")
print(f"   🌐 Player web: http://{SRS_SERVER_IP}:8080/players/srs_player.html?stream=bev_processed")
print(f"   📱 HLS: http://{SRS_SERVER_IP}:8080/hls/live/bev_processed.m3u8")

print(f"\n⚠️ Pressione Ctrl+C para parar antecipadamente")
print(f"🔄 Processamento começando...")

# Executar pipeline
try:
    success = pipeline.run_from_config(
        source=INPUT_SOURCE,
        max_frames=MAX_FRAMES
    )
    
    if success:
        print(f"\n🎉 Pipeline executado com sucesso!")
        
        # Mostrar estatísticas finais
        stats = pipeline.get_stats()
        print(f"\n📊 ESTATÍSTICAS FINAIS:")
        print(f"   🎬 Frames processados: {stats.get('processed_frames', 0)}")
        print(f"   🔍 Estimativas de profundidade: {stats.get('depth_estimations', 0)}")
        print(f"   🗺️ Vistas BEV geradas: {stats.get('bev_generations', 0)}")
        print(f"   📦 Detecções totais: {stats.get('detections', 0)}")
        print(f"   ⚡ FPS médio: {stats.get('fps', 0):.1f}")
        print(f"   ⏱️ Tempo médio por frame: {stats.get('processing_time_ms', 0):.1f}ms")
        
    else:
        print(f"\n⚠️ Pipeline interrompido ou com erro")
        print(f"💡 Verifique logs para mais detalhes")
        
except KeyboardInterrupt:
    print(f"\n⚠️ Pipeline interrompido pelo usuário")
    
except Exception as e:
    print(f"\n❌ Erro durante execução: {e}")
    print(f"💡 Verifique configurações e conectividade")
    
finally:
    # Limpeza
    if 'pipeline' in locals():
        pipeline.cleanup()
    print(f"🧹 Recursos limpos")


In [ ]:
# Configurações personalizadas de câmera
CUSTOM_CAMERA_CONFIG = {
    "focal_length_x": 800.0,         # Ajuste conforme sua câmera
    "focal_length_y": 800.0,         # Pode ser diferente de focal_x
    "principal_point_x": 320.0,      # Centro ótico X (geralmente width/2)
    "principal_point_y": 240.0,      # Centro ótico Y (geralmente height/2)
    "camera_height_meters": 1.6,     # Altura da câmera em metros
    "camera_tilt_degrees": 0.0,      # Inclinação da câmera (-90 a +90 graus)
}

# Configurações personalizadas de BEV
CUSTOM_BEV_CONFIG = {
    "range_meters": 50.0,            # Alcance da vista BEV
    "resolution_meters": 0.1,        # Resolução por pixel (menor = mais detalhado)
    "output_size": (500, 500),       # Tamanho da imagem BEV
    "center_offset": (250, 400),     # Posição da câmera na imagem BEV
}

print("⚙️ Configurações personalizadas definidas!")
print(f"📏 Configuração de câmera:")
for key, value in CUSTOM_CAMERA_CONFIG.items():
    print(f"   {key}: {value}")

print(f"\n🗺️ Configuração BEV:")
for key, value in CUSTOM_BEV_CONFIG.items():
    print(f"   {key}: {value}")

print(f"\n💡 Para aplicar, execute a próxima célula")


In [ ]:
# Criar pipeline com configuração personalizada
custom_pipeline = PseudoLiDARPipeline(
    camera_config=CUSTOM_CAMERA_CONFIG,
    bev_config=CUSTOM_BEV_CONFIG
)

# Testar diferentes modelos de profundidade
DEPTH_MODELS = {
    "fast": "MiDaS_small",          # Mais rápido, menor precisão
    "balanced": "MiDaS",            # Balanceado
    "quality": "DPT_Hybrid",        # Boa qualidade
    "best": "DPT_Large",            # Máxima qualidade, mais lento
}

# Escolha o modelo (recomendado: fast ou balanced para testes)
SELECTED_CUSTOM_MODEL = "balanced"

print(f"📋 Modelos disponíveis:")
for key, model in DEPTH_MODELS.items():
    print(f"   {key}: {model}")

print(f"\n🤖 Carregando modelo personalizado: {SELECTED_CUSTOM_MODEL}")
if custom_pipeline.load_models(model_name=DEPTH_MODELS[SELECTED_CUSTOM_MODEL]):
    print(f"✅ Modelo {SELECTED_CUSTOM_MODEL} carregado!")
    
    # Configurar streaming personalizado
    CUSTOM_OUTPUT_URL = f"rtmp://{SRS_SERVER_IP}:{SRS_RTMP_PORT}/live/bev_custom"
    
    if custom_pipeline.setup_streaming(CUSTOM_OUTPUT_URL):
        print(f"✅ Pipeline personalizado pronto!")
        print(f"📺 Stream personalizado: {CUSTOM_OUTPUT_URL}")
        print(f"🌐 Player: http://{SRS_SERVER_IP}:8080/players/srs_player.html?stream=bev_custom")
    else:
        print("⚠️ Streaming não configurado para pipeline personalizado")
        
else:
    print(f"❌ Erro ao carregar modelo {SELECTED_CUSTOM_MODEL}")

# Para usar o pipeline personalizado, substitua 'pipeline' por 'custom_pipeline' na execução


In [ ]:
# Mostrar informações do sistema
import sys
import torch
import cv2

print("🖥️ Informações do Sistema:")
print(f"   🐍 Python: {sys.version.split()[0]}")
print(f"   🔥 PyTorch: {torch.__version__}")
print(f"   📷 OpenCV: {cv2.__version__}")

try:
    import transformers
    print(f"   🤖 Transformers: {transformers.__version__}")
except:
    print(f"   ❌ Transformers: Não instalado")

if torch.cuda.is_available():
    print(f"   🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"   💾 Memória GPU Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    allocated = torch.cuda.memory_allocated() / 1024**3
    cached = torch.cuda.memory_reserved() / 1024**3
    print(f"   🔥 Memória GPU Usada: {allocated:.1f} GB (cache: {cached:.1f} GB)")
    print(f"   ⚡ Memória GPU Livre: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1024**3:.1f} GB")
else:
    print("   💻 Usando CPU (sem GPU disponível)")

print(f"\n📋 URLs de Acesso aos Streams:")
print(f"   🌐 Interface SRS: http://{SRS_SERVER_IP}:8080")
print(f"   📺 Player BEV: http://{SRS_SERVER_IP}:8080/players/srs_player.html?stream=bev_processed")
print(f"   📺 Player YOLO: http://{SRS_SERVER_IP}:8080/players/srs_player.html?stream=processed")
print(f"   📱 HLS BEV: http://{SRS_SERVER_IP}:8080/hls/live/bev_processed.m3u8")
print(f"   📱 HLS YOLO: http://{SRS_SERVER_IP}:8080/hls/live/processed.m3u8")

print(f"\n🔄 Comparação de Streams:")
print(f"   YOLO: rtmp://{SRS_SERVER_IP}:1935/live/processed (detecções 2D)")
print(f"   BEV:  rtmp://{SRS_SERVER_IP}:1935/live/bev_processed (vista 3D)")

print(f"\n📊 Performance Esperada:")
print(f"   🎯 YOLO: 25+ FPS, 44MB GPU, detecções rápidas")
print(f"   🗺️ Pseudo-LiDAR: 15 FPS, 1-2GB GPU, mapeamento 3D")
print(f"   ✅ Podem executar simultaneamente!")

print(f"\n🎮 Como usar:")
print(f"   1. Execute células de configuração e inicialização")
print(f"   2. Execute célula de 'Execução Principal'")
print(f"   3. Acesse URLs acima para visualizar streams")
print(f"   4. Compare com stream YOLO se disponível")
